# MLFlow

Pour rappel, la documentation Python de MLFlow se trouve à l'adresse :


https://mlflow.org/docs/latest/python_api/index.html

Vous allez suivre plusieurs étapes (plusieurs fonctions) qui permettront de lancer des exécutions successives dans la dernière cellule.

## Modélisation 
### Importation des librairies

In [ ]:
import os

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import random

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

import mlflow
from mlflow.tracking import MlflowClient
from mlflow.models import infer_signature

import time

In [ ]:
import warnings
warnings.filterwarnings("ignore")

### Importation des données : Cancer

Récupérez le fichier cancer450.csv et générez en son dataframe.
Affiche- le pour vérifier la bonne importation des données.

In [ ]:
#**A COMPLETER**

### Split du jeu d'entrainement et de test

Séparez le jeu de données en entraînement et test à 80% et une graine de 42.
Pour rappel, la variable cible est "tumor".

In [ ]:
#**A COMPLETER**

### Liste des modèles

Nous allons travailler avec les modèles suivants : 
* DecisionTree
* LogisticRegression
* SVC
* GaussianNB
* KNeigbors

In [ ]:
liste = ['DT','LR','SVM','NB','KNN']

### Hyperparamètres

Pour chaque modèle, nous définissons les hyperparamètres dont certains avec de façon aléatoire afin de simuler différentes exécutions.

In [ ]:
def hyperparametre(modele):
    if modele == 'DT':
        params = {
            "criterion": 'gini',  # La fonction pour mesurer la qualité d'une scission. Supporté : "gini" pour l'impureté de Gini, "entropy" pour le gain d'information.
            "splitter": 'best',  # La stratégie utilisée pour choisir la scission à chaque nœud. Supporté : "best" pour choisir la meilleure scission, "random" pour choisir la meilleure scission aléatoire.
            "max_depth": None,  # La profondeur maximale de l'arbre. Si None, alors les nœuds sont étendus jusqu'à ce que toutes les feuilles soient pures ou jusqu'à ce que toutes les feuilles contiennent moins que min_samples_split échantillons.
            "min_samples_split": random.randint(2, 50),  # Le nombre minimum d'échantillons requis pour scinder un nœud interne.
            "min_samples_leaf": random.randint(1, 2),  # Le nombre minimum d'échantillons requis pour être à un nœud feuille.
            "min_weight_fraction_leaf": 0.,  # La fraction pondérée minimale de la somme totale des poids (de tous les échantillons d'entrée) requise pour être à un nœud feuille.
            "max_features": None,  # Le nombre de fonctionnalités à considérer lors de la recherche de la meilleure scission. Si None, alors max_features=n_features.
            "random_state": None,  # Contrôle la randomicité de l'estimateur. Les valeurs de l'état aléatoire différentes peuvent changer le comportement de l'arbre.
            "max_leaf_nodes": None,  # Développe un arbre avec max_leaf_nodes de la meilleure façon. Si None, alors un nombre illimité de nœuds feuilles.
            "min_impurity_decrease": 0.,  # Un nœud sera scindé si cette scission induit une diminution de l'impureté supérieure ou égale à cette valeur.
            "class_weight": None,  # Poids associés aux classes. Si None, toutes les classes sont censées avoir un poids un.
            "ccp_alpha": 0.0,  # Paramètre de complexité utilisé pour la taille minimale de l'arbre de coût-complexité. Plus grande est la valeur de alpha, plus le nombre de nœuds est réduit.
        }
    elif modele == 'LR':
        params = {
            "penalty": 'l2',  # Spécifie la norme utilisée dans la pénalisation. Les valeurs les plus courantes sont 'l2' et 'l1'.
            "dual": False,  # Formulation duale ou primale. La formulation duale est seulement implémentée pour la pénalité 'l2' avec des solveurs 'liblinear'. Préférer False dans la majorité des cas.
            "tol": 1e-4,  # Tolérance pour les critères d'arrêt.
            "C": 1.0,  # Inverse de la force de régularisation; doit être un flottant positif. Comme dans les machines à vecteurs de support, des valeurs plus petites spécifient une régularisation plus forte.
            "fit_intercept": True,  # Spécifie si une constante (a.k.a. biais ou intercept) doit être ajoutée à la fonction de décision.
            "intercept_scaling": random.randint(1, 5),  # Utile seulement lorsque le solveur 'liblinear' est utilisé et 'fit_intercept' est défini à True. Dans ce cas, x devient [x, self.intercept_scaling], c'est-à-dire une colonne "synthétique" de poids égaux à intercept_scaling est ajoutée à l'instance vectorielle x.
            "class_weight": None,  # Poids associés aux classes. Si non spécifié, toutes les classes ont un poids un.
            "random_state": None,  # Le seed du générateur de nombres pseudo-aléatoires à utiliser lors de la mélange des données.
            "solver": 'lbfgs',  # Algorithme à utiliser dans le problème d'optimisation. Pour les petits ensembles de données, 'liblinear' est un bon choix, tandis que 'sag' et 'saga' sont plus rapides pour les grands.
            "max_iter": random.randint(1, 1000),  # Nombre maximal d'itérations prises pour que les solveurs convergent.
            "multi_class": 'auto',  # Si le choix 'auto', le choix du binaire ou de l'un contre le reste dépend du type de données et du solveur.
            "verbose": random.randint(1, 10),  # Pour le solveur 'liblinear' et 'lbfgs', définir verbose à tout nombre positif pour la verbosité.
            "warm_start": False,  # Lorsqu'il est défini à True, réutilise la solution de l'appel précédent pour s'adapter comme initialisation, sinon, efface simplement la solution précédente.
            "n_jobs": None,  # Nombre de cœurs CPU à utiliser lors de la parallélisation sur des classes. 'None' signifie 1 sauf dans un contexte joblib.parallel_backend. '-1' signifie utiliser tous les processeurs.
            "l1_ratio": None,  # Le ratio de mélange L1, uniquement utilisé si penalty='elasticnet'. 'l1_ratio=0' correspond à une pénalité L2, 'l1_ratio=1' à une L1. Pour '0 < l1_ratio < 1', la pénalité est une combinaison de L1 et L2.
        }
    elif modele == 'SVM':
        params = {
            "C": 1.0,  # Paramètre de régularisation. La force de la régularisation est inversement proportionnelle à C. Doit être strictement positif. La pénalité est une norme au carré l2.
            "kernel": 'rbf',  # Spécifie le type de noyau à utiliser dans l'algorithme. Il doit être 'linear', 'poly', 'rbf', 'sigmoid', 'precomputed' ou un appelable.
            "degree": random.randint(1, 5),  # Degré de la fonction du noyau polynomial ('poly'). Ignoré par tous les autres noyaux.
            "gamma": 'scale',  # Coefficient du noyau pour 'rbf', 'poly' et 'sigmoid'. Si 'gamma'='scale' (par défaut) est passé alors il utilise 1 / (n_features * X.var()) comme valeur de gamma, si 'auto', utilise 1 / n_features.
            "coef0": 0.0,  # Terme indépendant dans la fonction du noyau. C'est seulement significatif dans 'poly' et 'sigmoid'.
            "shrinking": True,  # Utilise la heuristique de shrinking ou non.
            "probability": False,  # Si vrai, active les estimations de probabilité, ce qui ralentit cette méthode.
            "tol": 1e-3,  # Tolérance pour le critère d'arrêt.
            "cache_size": random.randint(1, 1000),  # Taille du cache du noyau (en Mo).
            "class_weight": None,  # Poids associés aux classes dans le format {classe_label: poids}. Si non spécifié, toutes les classes ont un poids un.
            "verbose": False,  # Active la sortie verbosité.
            "max_iter": -1,  # Limite stricte sur les itérations au sein du solveur, ou -1 pour aucune limite.
            "decision_function_shape": 'ovr',  # Si 'ovr', renvoie la fonction de décision one-vs-rest (n_classes, n_samples) comme toutes les autres classificateurs. Si 'ovo', la fonction de décision one-vs-one (libsvm) est renvoyée (n_classes * (n_classes - 1) / 2, n_samples). Cependant, on peut toujours utiliser `one-vs-rest` en passant 'ovr' à l'option `decision_function_shape` du classificateur `OneVsRestClassifier` explicitement.
            "break_ties": False,  # Si vrai, la décision de 'decision_function_shape'='ovr', et le nombre de classes > 2, la prédiction brisera les liens selon les valeurs de la fonction de décision. Sinon, le premier parmi les classes liées sera renvoyé.
            "random_state": None,  # Le seed du générateur de nombres pseudo-aléatoires utilisé lors du mélange des données pour les probabilités d'estimation. Ignoré lorsque 'probability' est False.
        }
    elif modele == 'NB':
        params = {
            "var_smoothing": 1e-9,  # Portion de la variance la plus grande de toutes les caractéristiques qui est ajoutée à la variance pour la stabilité du calcul.
        }
    elif modele == 'KNN':
        params = {
            "n_neighbors": random.randint(1, 10),  # nombre de voisins à utiliser
            "weights": 'uniform',  # poids des points, peut être 'uniform' ou 'distance' ou une fonction personnalisée
            "algorithm": 'auto',  # algorithme utilisé pour calculer les voisins les plus proches, peut être 'auto', 'ball_tree', 'kd_tree', 'brute'
            "leaf_size": random.randint(1, 30),  # taille de la feuille passée à BallTree ou KDTree
            "p": 2,  # paramètre de puissance pour la métrique Minkowski
            "metric": 'minkowski',  # la métrique de distance à utiliser pour l'arbre
            "metric_params": None,  # arguments supplémentaires pour la métrique de distance
            "n_jobs": None,  # nombre de travaux parallèles à exécuter pour la recherche de voisins
        }
    
    return params

### Entraînement et prédiction du modèle

La fonction suivante **entrainement_prediction(n)** est défini comme suit :
- ENTREE : name_model qui représente le nom du modèle
- SORTIE : retourne le modèle et les prédictions

Affecter chaque modèle avec la fonction sklearn qui lui est propre en prenant en compte les hyperparamètres (fonction précédente : hyperparametre(modele)).

In [ ]:
def entrainement_prediction(name_model):
    #**A COMPLETER**
    return modele, modele.predict(X_test)

### Calculs des metriques

Calculez-les métriques suivantes sous forme de fonction :
* **accuracy(y_test, y_pred)**
* **precision(y_test, y_pred)**
* **recall(y_test, y_pred)**
* **f1(y_test, y_pred)**

In [ ]:
#**A COMPLETER**

### Matrice de confusion

Cette fonction **conf_matrix(y_test, y_pred)** créé la matrice de confusion, la génère avec matplotlib.pyplot, l'enregistre dans le fichier confusion_matrix.png et retourne le chemin de l'image enregistré sur le disque.

In [ ]:
#**A COMPLETER**
def conf_matrix(y_test, y_pred):
    # Matrice de confusion

    # Génération de la matrice de confusion avec matplotlib
    
    # Sauvegarde de la figure

    return fig_path

## Trace des expériences

Le serveur de suivi (tracking server) est utilisé pour enregistrer et consulter les métriques, les paramètres et les artefacts associés aux différentes exécutions de votre programme de machine learning (expériences). 

Dans votre environnement et dans une console, démarrez le serveur MLFlow (cherchez dans la documentation) et vérifiez que le serveur est actif en saisissant l'adresse `http://127.0.0.1:5000` dans un navigateur.

### Définition de l'URI du serveur

Spécifiez l'URI du tracking server en local sur le port 5000.

Vous trouverez l'information dans la documentation Python de MLFlow :
https://www.mlflow.org/docs/latest/getting-started/intro-quickstart/index.html

In [ ]:
#**A COMPLETER**

### Création d'une nouvelle experience

L'expérience va vous servir à organiser toutes vos exécutions. 

Appelez-la "Experience_cancer" (set_experiment).

In [ ]:
#**A COMPLETER**

### Suivi des exécutions

Lancez la cellule suivante et observez le résultat sur l'interface MLFlow.

Il vous faut compléter au fur et à mesure la cellule tout en relançant l'exécution entre chaque étape ci-dessous pour observer le résultat :

1. Récupérez les **prédictions** à partir de la fonction `entrainement_prediction()` et affectez-les à une variable `y_pred`
2. Récupérez les **métriques de log** (voir `log_system_metrics` et `time.sleep`)
   https://mlflow.org/docs/latest/system-metrics/index.html
3. Chargez les **hyperparamètres** avec la fonction `hyperparametre()` (voir `log_param`)
4. Ajoutez des **tags** d'information sur le modèle utilisé, par exemple (voir `set_tag`)
5. Chargez les **métriques** dans MLFlow (voir `log_metric`)
6. Chargez la **matrice de confusion** comme artefact avec la fonction précédente `conf_matrix()` (voir `log_artifact`)
7. Chargez les **données** comme artefact (voir `log_artifact`)
8. Ajoutez le **dataset** de test dans l'interface des expériences (à l'identique que celui d'entrainement)
9. Chargez le **modèle** dans la variable `sk_model` à l'aide de la fonction `entrainement_prediction()`

In [ ]:
for i in range(1):
    for l in liste:
        print(l)
        ##########**A COMPLETER**########## 1. Récupération des prédicions

        run_description = """
        Le jeu de données "Breast Cancer Wisconsin (Diagnostic) Data Set" est un ensemble de données largement utilisé dans le domaine de l'apprentissage automatique pour la classification binaire, en particulier pour la prédiction du diagnostic de cancer du sein (maligne vs bénigne). 
        Il a été créé par le Dr. William H. Wolberg à l'Université du Wisconsin-Madison.


        L'ensemble de données comprend des caractéristiques calculées à partir d'images numérisées de biopsie de tissus mammaires. 
        Ces caractéristiques décrivent les noyaux cellulaires présents dans les images et sont calculées à partir d'une image numérisée d'une ponction à l'aiguille fine (FNA) d'une masse mammaire. 
        Elles incluent des mesures telles que le rayon moyen des cellules, la texture, la périmétrie, l'aire, la douceur, la compacité, la concavité, les points concaves, la symétrie, et la dimension fractale.
        """
    
        tags = {
            'mlflow.note.content': run_description
        }

        ##########**A COMPLETER**########## 2. Collecte des metriques de log
        with mlflow.start_run(tags=tags) as run:
            
            # Chargement des métadonnées
            mlflow.log_param("Nombre de lignes", X.shape[0])
            mlflow.log_param("Nombre de caractéristiques", X.shape[1])
            
            ##########**A COMPLETER**########## 3. Chargement des Hyperparametres

            ##########**A COMPLETER**########## 4. Definition des tags
        
            ##########**A COMPLETER**########## 5. Chargement des metriques

            ##########**A COMPLETER**########## 6. Chargement de la matrice de confusion en temps qu'artefact
            
            ##########**A COMPLETER**########## 7. Chargement des donnees (Sauvegarde dans un fichie CSV, Enregistrement comme artefact)

            dataset_training = mlflow.data.from_pandas(X_train,
              source="file_path",
              name="Cancer"
            )
            mlflow.log_input(dataset_training, context="training")

            ##########**A COMPLETER**########## 8. Ajout du dataset de test dans l'interface des expériences
        
            # Signature
            #signature = infer_signature(X_train, y_pred)
        
            #model_info = mlflow.sklearn.log_model(
                #sk_model= ##########**A COMPLETER**########## 9. Chargement du modèle
                #artifact_path="Cancer_Model",
                #signature=signature,
                #input_example=X_train,
            #)
        
            eval_data = pd.DataFrame(X_test)
            eval_data["tumor"] = y_test.tolist()
        
        print(mlflow.MlflowClient().get_run(run.info.run_id).data)